# Day 7

## Part 1
- commands are prepended with `$`
- `cd` is cd
- `cd /` goes back to the top directory
- `ls` is ls
    - files are printed as: `<size> <name>`
    - directories are printed as: `dir <name>`

`Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?`

- Directory sizes are counted independently so the final total can contain the same file several times

In [26]:
from utils import parse_from_file, ParseConfig

# using this parser means we get blocks of output where the first lines are the
# ls printout and then at the end there are directory changes
parser = ParseConfig('\n$ ls\n', ParseConfig('\n', str))

terminal_output = parse_from_file('day_7.txt', parser)

print(terminal_output[:3])

[['$ cd /'], ['dir lhrfs', '193233 mvsjmrtn', 'dir nwh', 'dir pjsd', 'dir qfrrtb', '31987 zzdfcs', '$ cd lhrfs'], ['197903 hzl.jdj', '42249 wsbpzmbq.hws', '$ cd ..', '$ cd nwh']]


In [27]:
def change_directory(cd_command: str, current_dir: str) -> str:
    """
    returns the updated directory
    """
    arg = cd_command.replace('$ cd ', '')  # strip of the command
    if arg == '/':
        return '/'
    elif arg == '..':
        return '/'.join(current_dir.split('/')[:-2]) + '/'
    else:
        return current_dir + arg + '/'

test_dir = '/'
print(test_dir)
test_changes = ['abc', 'def', '..', '..', 'ghi', '/']
for arg in test_changes:
    test_dir = change_directory(arg, test_dir)
    print(test_dir)


/
/abc/
/abc/def/
/abc/
/
/ghi/
/


In [28]:
def get_file_system(terminal_output: 'list[list[str]]') -> dict:
    """
    maps out a file system from a terminal output
    """
    flat_map = {}
    current_dir = '/'
    for block in terminal_output:
        for line in block:
            if line.startswith('$'):
                current_dir = change_directory(line, current_dir)
                continue
            
            # else
            if current_dir not in flat_map:
                flat_map.update({current_dir: dict()})
            
            size, name = line.split(' ')
            try:
                size = int(size)
            except ValueError:
                pass
            flat_map[current_dir].update({name: size})

    return flat_map

flat_map = get_file_system(terminal_output)

print('\n'.join([str(item) for item in list(flat_map.items())[:10]]))

('/', {'lhrfs': 'dir', 'mvsjmrtn': 193233, 'nwh': 'dir', 'pjsd': 'dir', 'qfrrtb': 'dir', 'zzdfcs': 31987})
('/lhrfs/', {'hzl.jdj': 197903, 'wsbpzmbq.hws': 42249})
('/nwh/', {'bgrccm.tqh': 63077, 'dznccwl.bnw': 69961, 'pmdj': 'dir', 'rsbvj.jtd': 187013})
('/nwh/pmdj/', {'rlgfd.rrd': 292527, 'tbj.grn': 68737, 'wsbpzmbq.hws': 153072})
('/pjsd/', {'czzcslm': 'dir', 'dgwpl': 'dir', 'fqg': 'dir', 'lszhdjr': 'dir', 'mmpf': 'dir', 'wtwhzzwz': 'dir', 'zzdfcs': 149748})
('/pjsd/czzcslm/', {'bvrnzhd.vzp': 249237, 'ssvqllt.ccv': 16960})
('/pjsd/dgwpl/', {'brsbfqbm.hls': 23547, 'ljzrwpv': 'dir'})
('/pjsd/dgwpl/ljzrwpv/', {'btnzjtlr': 'dir', 'czr': 'dir'})
('/pjsd/dgwpl/ljzrwpv/btnzjtlr/', {'tbj.mwg': 191998})
('/pjsd/dgwpl/ljzrwpv/czr/', {'fqg': 'dir'})


In [29]:
def get_dir_size(directory: str, size_lookup: dict, full_set: dict) -> int:
    """
    finds the size of a directory using the size look up first if possible
    """
    if directory in size_lookup:
        return size_lookup[directory]
    # else
    dir_size = 0
    for sub_dir, size in full_set[directory].items():
        if isinstance(size, int):
            dir_size += size
        else:
            dir_size += get_dir_size(
                directory + sub_dir + '/', size_lookup, full_set)

    return dir_size
    
for path in ['/lhrfs/', '/nwh/']:
    print(f'{path} is {get_dir_size(path, {}, flat_map)} big!')

/lhrfs/ is 240152 big!
/nwh/ is 834387 big!


In [30]:
def find_total_sizes(flat_map: 'dict[dict]') -> dict:
    """
    returns a dictionary of the total sizes of all the directories

    uses memoisation!
    """
    dir_sizes = {}
    for directory, contents in flat_map.items():
        dir_sizes.update(
            {directory: get_dir_size(directory, dir_sizes, flat_map)})
    
    return dir_sizes

dir_sizes = find_total_sizes(flat_map)

In [31]:
# ok finally let's sum the totals!
sum_under_100k = sum([size for size in dir_sizes.values() if size < 100000])

print(
    'the sum of directories\'s sizes with size less than 100k is '
    f'{sum_under_100k}')

the sum of directories's sizes with size less than 100k is 1243729


## Part 2

- 30,000,000 is required to update
- the total file system has 70,000,000

`Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?`

In [34]:
total_space = 70000000

remaining_space = total_space - dir_sizes['/']

update_requirement = 30000000

minimum_size_to_delete = update_requirement - remaining_space

print(f'used storage: {dir_sizes["/"]}/{total_space}')

print(f'remaining space: {remaining_space}')

print(f'minimum storage required to be freed up: {minimum_size_to_delete}')

used storage: 44376732/70000000
remaining space: 25623268
minimum storage required to be freed up: 4376732


In [36]:
dir_to_delete = '/'
for directory, size in dir_sizes.items():
    if minimum_size_to_delete <= size < dir_sizes[dir_to_delete]:
        dir_to_delete = directory

print(f'the directory to delete is: {directory}')
print(f'its size is: {dir_sizes[dir_to_delete]}')

the directory to delete is: /qfrrtb/qlnwhq/sdhqp/
its size is: 4443914
